In [1]:
# setup cell!!

from canvasapi import Canvas
from canvasapi.exceptions import BadRequest
from datetime import datetime, time, timedelta
from zoneinfo import ZoneInfo
import ipywidgets as widgets
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
from thefuzz import fuzz, process

# setup canvas connection
tokenfile = open("canvas-token", "r")
token = tokenfile.readline()
tokenfile.close()
url = "https://westminster.instructure.com/"
canvas = Canvas(url, token)

# now i'ma hardcode the courses that people will ask for extensions in
# this will avoid a number of (slow!) requests and filters.
math202id = 3387585
math202 = canvas.get_course(math202id)
phys309id = 3388078
phys309 = canvas.get_course(phys309id)
coursedict = {
    "MATH 202": math202,
    "PHYS 309": phys309
}

# setup google connection
# If modifying these scopes, delete the file google-token.json.
scopes = ['https://www.googleapis.com/auth/spreadsheets.readonly']
spreadsheet_id = '1mtc172TQIju8BwAiqwFS8Fwtz1I8ZN6Aw7SwfrVvdG0'
data_range = 'B2:I' #this probably works for now

creds = None
# The file google-token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first time.
if os.path.exists('google-token.json'):
    creds = Credentials.from_authorized_user_file('google-token.json', scopes)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'google-credentials.json', scopes)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('google-token.json', 'w') as token:
        token.write(creds.to_json())


In [2]:
# utility function: combine two PaginatedLists
def combine_pl(pl1, pl2):
    # iterate through both to build the elements lists
    i = 0
    dellist = []
    for item in pl1: pass
    for item in pl2: pass
    pl1._elements = pl1._elements + pl2._elements
    # remove assignments that aren't published
    return pl1

In [3]:
# utility function: when I have an assignment list, 
# I'd like to throw away the unpublished ones.
def remove_unpublished(pl):
    i = 0
    dellist = []
    for item in pl: 
        # keep track of not-published items
        if not item.published: dellist.append(i)
        i+=1
    # remove assignments that aren't published
    for i in sorted(dellist, reverse=True):
        del pl._elements[i]
    return pl

In [4]:
# I will also pre-pull the assignment lists and do a little pre-filtering.
# in math 202 specifically, I only care about problem sets and webwork
# so let's grab those by groups
for group in math202.get_assignment_groups(): 
    if group.name == "Problem Sets":
        math202_assgs = math202.get_assignments_for_group(group.id)
    elif group.name == "Webwork":
        math202_webwork = math202.get_assignments_for_group(group.id)

math202_assgs = remove_unpublished(combine_pl(math202_assgs, math202_webwork))

phys309_assgs = remove_unpublished(phys309.get_assignments())


In [8]:
# read data from spreadsheet
try:
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id,
                                range=data_range).execute()
    requests = result.get('values', [])
    requestsdf = pd.DataFrame(requests, columns = ["name", "email", "classname", 
        "assignment", "due date", "need1", "need2", "status"])
    # pull out the ones that I need to do something with
    # tododf = requestsdf[pd.isna(requestsdf["status"])]
    tododf = requestsdf

    if not values:
        print('No data found.')
except HttpError as err:
    print(err)

In [9]:
# utility function: given a list of assignments
# and a string that a student wrote,
# I'm going to create a dropdown of possibilities,
# sorted by matching score,
# and pre-populated with my best guess.
def find_assignment(assglist, inputstr):
    namesdict = {}
    for assg in assglist:
        namesdict[assg.name]=assg
    # calculate and sort by matching score: 
    fuzzresults = process.extract(inputstr, namesdict.keys())
    opts = [] 
    for result in fuzzresults:
        opts.append( (result[0], namesdict[result[0]]) )
        # this is now a list of tuples like widgets.Dropdown wants.
    dropdown = widgets.Dropdown(
        options=opts,
        # value=None, -- I'm going to prepopulate with my best guess.
        description="Probably: ",
        disabled=False,
    )
    return dropdown

In [11]:
output = widgets.Output()
dropdown = find_assignment(math202_assgs, "Problem Set 3")
display(dropdown, output)
def on_value_change(change):
    with output:
        output.clear_output()
        print(change['new'])

dropdown.observe(on_value_change, names = 'value')

Dropdown(description='Probably: ', options=(('PS#3: Integration by substitution and integration by parts', Ass…

Output()

In [2]:
def build_override_dict(inlist):
    [name, email, coursename, assignment, dd, need1, need2, status] = inlist

    # find the correct course and lookup the student
    course = coursedict[coursename]
    try: student = course.get_users(search_term = email)[0]
    except IndexError as err:
        print("haha that email's wrong")
        return
    
    # parse the date correctly as a datetime object
    # comes in from google forms as, e.g., "1/30"
    duetime = timedelta(hours=23,minutes=59)
    duedate = datetime.strptime(dd+"/23", '%m/%d/%y').replace(tzinfo=ZoneInfo("America/Denver"))
    duedate = duedate + duetime

    return {
        'student_ids': [student.id],
        'due_at': duedate, 
        'all_day': True,
        'all_day_date':  str(duedate.date())
    }

In [ ]:
# so now what I am going to do is something like, 
# filter the requestsdf to only have the todo ones
# then tododf.apply(build_override_dict, axis=1) 

# at some point we also need to do the assignment lookup
# and finally commit all of those overrides to canvas.

In [2]:
emailw = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)
coursew = widgets.Dropdown(
    options=[("MATH 202", math202), ("PHYS 309", phys309)],
    #value='MATH 202',
    description='Course:',
    disabled=False,
)
display(emailw)
display(coursew)

Text(value='Hello World', description='String:', placeholder='Type something')

Dropdown(description='Course:', options=(('MATH 202', Course(_requester=<canvasapi.requester.Requester object …

In [ ]:
# hey so this is 74% of the body of the send request function
try:
    test_assg.create_override(
        assignment_override={
            'student_ids': [12261567],
            'due_at': datetime(2023,2,1,23,59,59,tzinfo=ZoneInfo('America/Denver')),
            'all_day': True,
            'all_day_date':  '2023-02-01'
        }
    )
except BadRequest: # I need to be a little more careful here.
    # It's possible that the student is *one* of several students in an override.
    # If so, I need to remove them from the old override
    # and create them a new override
    # (otherwise it might fuck up other people's overrides!)
    bad_override = next((ov for ov in test_assg.get_overrides() if 12261567 in ov.student_ids), None)
    bad_override.edit(
        assignment_override={
            'student_ids': [12261567],
            'due_at': datetime(2023,2,1,23,59,59,tzinfo=ZoneInfo('America/Denver')),
            'all_day': True,
            'all_day_date':  '2023-02-01'
        }
    )


In [ ]:
test_assg.create_override(
    assignment_override={
        'student_ids': [12273191],
        'due_at': datetime(2023,2,1,23,59,59,tzinfo=ZoneInfo('America/Denver')),
        'all_day': True,
        'all_day_date':  '2023-02-01'
    }
)